<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import" data-toc-modified-id="Import-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import</a></span></li><li><span><a href="#Pre-Processing" data-toc-modified-id="Pre-Processing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Pre-Processing</a></span></li><li><span><a href="#Custom-Dataset" data-toc-modified-id="Custom-Dataset-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Custom Dataset</a></span></li><li><span><a href="#Model" data-toc-modified-id="Model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Model</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Inference" data-toc-modified-id="Inference-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Inference</a></span></li><li><span><a href="#Submission" data-toc-modified-id="Submission-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Submission</a></span></li></ul></div>

## Import

In [1]:
import random
import os

import numpy as np
import pandas as pd

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from rdkit import DataStructs
from rdkit.Chem import PandasTools, AllChem, rdMolDescriptors, MolFromSmiles



In [2]:
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')

In [3]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)

seed_everything(42) # Seed 고정

## Pre-Processing

In [4]:
from mendeleev.fetch import fetch_table
from rdkit import Chem
from rdkit.Chem import (Descriptors,
                        Lipinski,
                        Crippen,
                        rdMolDescriptors,
                        MolFromSmiles,
                        AllChem,
                        PandasTools)

from torch import Tensor
from torch_geometric.utils.sparse import dense_to_sparse

from transformers import AutoTokenizer, AutoModel

import numpy as np

from sklearn.preprocessing import StandardScaler
from rdkit import DataStructs
from deepchem import feat

archieve = ['seyonec/PubChem10M_SMILES_BPE_450k', "DeepChem/ChemBERTa-77M-MTR", 'seyonec/ChemBERTa-zinc-base-v1', 'seyonec/ChemBERTa_zinc250k_v2_40k']
chosen = archieve[2]

class Chemical_feature_generator():
    def __init__(self) -> None:
        mendeleev_atomic_f = ['atomic_radius', 'atomic_radius_rahm', 'atomic_volume', 'atomic_weight', 'c6', 'c6_gb', 
                        'covalent_radius_cordero', 'covalent_radius_pyykko', 'covalent_radius_pyykko_double', 'covalent_radius_pyykko_triple', 
                        'density', 'dipole_polarizability', 'dipole_polarizability_unc', 'electron_affinity', 'en_allen', 'en_ghosh', 'en_pauling', 
                        'heat_of_formation', 'is_radioactive', 'molar_heat_capacity', 'specific_heat_capacity', 'vdw_radius']
                        # Others are fine
                        # Heat of Formation: This reflects the energy associated with the formation of a molecule and might indirectly impact metabolic reactions.
                        # Is Radioactive: This binary property may not be directly relevant to metabolic stability.
                        # Molar Heat Capacity, Specific Heat Capacity: These properties relate to heat transfer but might not be directly tied to metabolic stability.
        self.mendeleev_atomic_f_table = fetch_table('elements')[mendeleev_atomic_f]
        
        self.DMPNNFeaturizer = feat.DMPNNFeaturizer()
        self.Mol2VecFingerprint = feat.Mol2VecFingerprint()
        self.BPSymmetryFunctionInput = feat.BPSymmetryFunctionInput(max_atoms=150)
        
        
        self.tokenizer = AutoTokenizer.from_pretrained(chosen)
        

    def get_atomic_features(self,atom):

        atomic_num = atom.GetAtomicNum() - 1 # -1 is offset
        mendel_atom_f = self.mendeleev_atomic_f_table.loc[atomic_num]

        mendel_atom_f = mendel_atom_f.to_numpy().astype(np.float32)

        rdkit_atom_f = [atom.GetDegree(),
                atom.GetTotalDegree(),
                atom.GetFormalCharge(),
                atom.GetIsAromatic()*1.,
                atom.GetNumImplicitHs(),
                atom.GetNumExplicitHs(),
                atom.GetTotalNumHs(),
                atom.GetNumRadicalElectrons(),
                atom.GetImplicitValence(),
                atom.GetExplicitValence(),
                atom.GetTotalValence(),
                atom.IsInRing()*1.]
        
        return mendel_atom_f, rdkit_atom_f
    
    def get_molecular_features(self, mol):
        ## 1. Molecular Descriptors 5
        MolWt = Descriptors.MolWt(mol)
        HeavyAtomMolWt = Descriptors.HeavyAtomMolWt(mol)
        NumValenceElectrons = Descriptors.NumValenceElectrons(mol)
        MolMR = Crippen.MolMR(mol)
        MolLogP = Crippen.MolLogP(mol)

        ## 2. Lipinski's Rule of Five 16
        FractionCSP3 = Lipinski.FractionCSP3(mol)
        HeavyAtomCount = Lipinski.HeavyAtomCount(mol)
        NHOHCount = Lipinski.NHOHCount(mol)
        NOCount = Lipinski.NOCount(mol)
        NumAliphaticCarbocycles = Lipinski.NumAliphaticCarbocycles(mol)
        NumAliphaticHeterocycles = Lipinski.NumAliphaticHeterocycles(mol)
        NumAliphaticRings = Lipinski.NumAliphaticRings(mol)
        NumAromaticCarbocycles = Lipinski.NumAromaticCarbocycles(mol)
        NumAromaticHeterocycles = Lipinski.NumAromaticHeterocycles(mol)
        NumAromaticRings = Lipinski.NumAromaticRings(mol)
        NumHAcceptors = Lipinski.NumHAcceptors(mol)
        NumHDonors = Lipinski.NumHDonors(mol)
        NumHeteroatoms = Lipinski.NumHeteroatoms(mol)
        NumRotatableBonds = Lipinski.NumRotatableBonds(mol)
        RingCount = Lipinski.RingCount(mol)
        CalcNumBridgeheadAtom = rdMolDescriptors.CalcNumBridgeheadAtoms(mol)

        ## 3. Additional Features 11
        ExactMolWt = Descriptors.ExactMolWt(mol)
        NumRadicalElectrons = Descriptors.NumRadicalElectrons(mol)
        NumSaturatedCarbocycles = Lipinski.NumSaturatedCarbocycles(mol)
        NumSaturatedHeterocycles = Lipinski.NumSaturatedHeterocycles(mol)
        NumSaturatedRings = Lipinski.NumSaturatedRings(mol)
        CalcNumAmideBonds = rdMolDescriptors.CalcNumAmideBonds(mol)
        CalcNumSpiroAtoms = rdMolDescriptors.CalcNumSpiroAtoms(mol)
        
        num_amion_groups = len(mol.GetSubstructMatches(MolFromSmiles("[NH2]")))
        num_ammonium_groups = len(mol.GetSubstructMatches(MolFromSmiles("[NH4+]")))
        num_sulfinic_acid_groups = len(mol.GetSubstructMatches(MolFromSmiles("[S](=O)(=O)")))
        num_alkoxy_groups = len(mol.GetSubstructMatches(MolFromSmiles('CO'))) 
        
        return [MolWt,
                HeavyAtomMolWt,
                NumValenceElectrons,
                FractionCSP3,
                HeavyAtomCount,
                NHOHCount,
                NOCount,
                NumAliphaticCarbocycles,
                NumAliphaticHeterocycles,
                NumAliphaticRings,
                NumAromaticCarbocycles,
                NumAromaticHeterocycles,
                NumAromaticRings,
                NumHAcceptors,
                NumHDonors,
                NumHeteroatoms,
                NumRotatableBonds,
                RingCount,
                MolMR,
                CalcNumBridgeheadAtom,
                ExactMolWt,
                NumRadicalElectrons,
                NumSaturatedCarbocycles,
                NumSaturatedHeterocycles,
                NumSaturatedRings,
                MolLogP,
                CalcNumAmideBonds,
                CalcNumSpiroAtoms,
#                 num_carboxyl_groups,
                num_amion_groups,
                num_ammonium_groups,
                num_sulfinic_acid_groups,
                num_alkoxy_groups]

    def get_molecule_fingerprints(self, mol):
        ECFP12 = AllChem.GetHashedMorganFingerprint(mol, 6, nBits=2048) # 2048
        ECFP6 = AllChem.GetHashedMorganFingerprint(mol, 3, nBits=2048) # 2048
        
        MACCS = Chem.rdMolDescriptors.GetMACCSKeysFingerprint(mol) # 167
        RDK_fp = Chem.RDKFingerprint(mol) # 2048
        Layer_fp = Chem.rdmolops.LayeredFingerprint(mol) # 2048
        Pattern_fp = Chem.rdmolops.PatternFingerprint(mol) # 2048
        
        ecfp12 = np.zeros((1,), dtype=np.int8)
        ecfp6 = np.zeros((1,), dtype=np.int8)
        maccs = np.zeros((1,), dtype=np.int8)
        rdk_fp = np.zeros((1,), dtype=np.int8)
        layer_fp = np.zeros((1,), dtype=np.int8)
        pattern_fp = np.zeros((1,), dtype=np.int8)
        DataStructs.ConvertToNumpyArray(ECFP12, ecfp12)
        DataStructs.ConvertToNumpyArray(ECFP6, ecfp6)
        DataStructs.ConvertToNumpyArray(MACCS, maccs)
        DataStructs.ConvertToNumpyArray(RDK_fp, rdk_fp)
        DataStructs.ConvertToNumpyArray(Layer_fp, layer_fp)
        DataStructs.ConvertToNumpyArray(Pattern_fp, pattern_fp)
        return np.hstack([ecfp12, ecfp6, maccs, rdk_fp, layer_fp, pattern_fp]) 
    
    def get_mol_feature_from_deepchem(self, smiles):
        return self.Mol2VecFingerprint(smiles) # (1, 300)
    
    def encoder_smiles(self, smiles):
        inputs = self.tokenizer.encode_plus(smiles, padding=True, return_tensors='pt', add_special_tokens=True)
        return inputs['input_ids']
    
    def get_atomic_feature_from_deepchem(self, smiles):

        DMPNN_F = self.DMPNNFeaturizer(smiles)[0].node_features
        
        return DMPNN_F

    def generate_mol_atomic_features(self, smiles):

        mol = Chem.MolFromSmiles(smiles)

        # gathering atomic feature 
        mendel_atom_features = [] 
        rdkit_atom_features = [] 
        for atom in mol.GetAtoms():
            mendel_atom_f, rdkit_atom_f = self.get_atomic_features(atom)

            mendel_atom_features.append(mendel_atom_f)
            rdkit_atom_features.append(rdkit_atom_f)

        dc_atmoic = self.get_atomic_feature_from_deepchem(smiles)
        atomic_features = np.concatenate([mendel_atom_features, rdkit_atom_features, dc_atmoic], axis=1, dtype=np.float32)
        
        return atomic_features
    
    def get_adj_matrix(self, smiles):
        
        mol = MolFromSmiles(smiles)
        adj = Chem.rdmolops.GetAdjacencyMatrix(mol)
        
        edge_index, edge_attr = dense_to_sparse(Tensor(adj))
            
        return edge_index, edge_attr


if __name__ == '__main__' : 
    import pandas as pd 
    from tqdm import tqdm
    from deepchem.feat.molecule_featurizers import RDKitDescriptors
    
    train = pd.read_csv('./train.csv')
    test = pd.read_csv('./test.csv')
    
    generator = Chemical_feature_generator()
    
    def process(df):
        molecular_f = [] 
        for sample in tqdm(df.SMILES):
            sample = Chem.MolFromSmiles(sample)
            molecular_features = generator.get_molecular_features(mol=sample)
            molecular_f.append(molecular_features)

        molecular_f = np.concatenate([molecular_f], axis=0)

    
        return pd.DataFrame(data=molecular_f, columns=['MolWt','HeavyAtomMolWt','NumValenceElectrons','FractionCSP3','HeavyAtomCount',
                                                   'NHOHCount','NOCount','NumAliphaticCarbocycles','NumAliphaticHeterocycles','NumAliphaticRings',
                                                   'NumAromaticCarbocycles','NumAromaticHeterocycles','NumAromaticRings','NumHAcceptors','NumHDonors',
                                                   'NumHeteroatoms','NumRotatableBonds','RingCount','MolMR','CalcNumBridgeheadAtom','ExactMolWt',
                                                   'NumRadicalElectrons','NumSaturatedCarbocycles','NumSaturatedHeterocycles','NumSaturatedRings','MolLogP',
                                                   'CalcNumAmideBonds','CalcNumSpiroAtoms','num_amion_groups','num_ammonium_groups',
                                                   'num_sulfinic_acid_groups','num_alkoxy_groups'])

    train_molecular_f = process(train)
    train_merged = pd.concat([train, train_molecular_f], axis=1)
    
    test_molecular_f = process(test)
    test_merged = pd.concat([test, test_molecular_f], axis=1)
    
    train_merged.to_csv('./new_train.csv', index=False)
    test_merged.to_csv('./new_test.csv', index=False)
    
    def deepchem_rdkit(df):
        
        featurizer = RDKitDescriptors()
        rdkit_features = []
        
        for smiles in tqdm(df.SMILES):
            feature = featurizer(smiles)
            rdkit_features.append(feature)
            
        return np.concatenate(rdkit_features)
    
    
    features = deepchem_rdkit(train)
    column_means = np.mean(features, axis=0)
    non_zero_mean_columns = np.where(column_means != 0)[0]
    features = features[:, non_zero_mean_columns]
    features = np.concatenate([np.reshape(train.SMILES, (-1, 1)), features], axis=1)
    features = pd.DataFrame(features).dropna(axis=1)
    pd.DataFrame(features).to_csv('./rdkit_train.csv', index=False)
    train_col = features.columns
    features = deepchem_rdkit(test)
    features = features[:, non_zero_mean_columns]
    features = np.concatenate([np.reshape(test.SMILES, (-1 ,1)), features], axis=1)
    features = pd.DataFrame(features).dropna(axis=1)
    features = features[train_col]
    pd.DataFrame(features).to_csv('./rdkit_test.csv', index=False)
    test_col = features.columns
    
    print(list(train_col), len(train_col))
    print(list(test_col), len(test_col))
    


    
    

Skipped loading some Jax models, missing a dependency. No module named 'jax'
100%|█████████████████████████████████████████| 483/483 [00:07<00:00, 63.23it/s]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 14, 15, 16, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191] 180
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 14, 15, 16, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62

## Custom Dataset

In [5]:
from typing import Callable, Optional, Union

import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.feature_selection import VarianceThreshold

import torch 
from torch.utils.data import Dataset

from torch_geometric.data import Data
from torch_geometric.data.dataset import IndexType
from torch_geometric.loader import DataLoader

import pytorch_lightning as pl

from rdkit.Chem import PandasTools

                 
feature_label = [ 'MolWt', 'HeavyAtomMolWt',
                    'NumValenceElectrons', 'FractionCSP3', 'HeavyAtomCount', 'NHOHCount',
                    'NOCount', 'NumAliphaticCarbocycles', 'NumAliphaticHeterocycles',
                    'NumAliphaticRings', 'NumAromaticCarbocycles',
                    'NumAromaticHeterocycles', 'NumAromaticRings', 'NumHAcceptors',
                    'NumHDonors', 'NumHeteroatoms', 'NumRotatableBonds', 'RingCount',
                    'MolMR', 'CalcNumBridgeheadAtom', 'ExactMolWt', 
                    'NumSaturatedCarbocycles', 'NumSaturatedHeterocycles',
                    'NumSaturatedRings', 'MolLogP', 'CalcNumAmideBonds',
                    'CalcNumSpiroAtoms',  
                    'num_ammonium_groups', 'num_sulfinic_acid_groups', 'num_alkoxy_groups'] # 30 


given_features = ['AlogP','Molecular_Weight','Num_H_Acceptors','Num_H_Donors','Num_RotatableBonds','LogD','Molecular_PolarSurfaceArea'] # 7 

generator = Chemical_feature_generator()

class Chemcial_dataset(Dataset):
    def __init__(self, 
                 data_frame:pd.DataFrame,
                 fps,
                 mol_f,
                 transform=None,
                 is_train=True):
        super().__init__()
        self.df = data_frame
        self.fps = fps
        self.mol_f = mol_f
        self.transform = transform
        self.is_train = is_train
            
        
    def __getitem__(self, idx):
        return self.get_chem_prop(idx)
        
    def __len__(self) -> int:
        return self.df.shape[0]
    
    def get_chem_prop(self, idx):
        
        sample = self.df.iloc[idx]
        fingerprint = self.fps[idx]
        molecular_feature = self.mol_f[idx]
        smiles = sample['SMILES']
        
        edge_index, edge_attr = generator.get_adj_matrix(smiles=smiles)
        atomic_feature = generator.generate_mol_atomic_features(smiles=smiles)
        input_ids = generator.encoder_smiles(smiles) # 384


        if self.is_train:
            MLM = sample['MLM']
            HLM = sample['HLM']
            atomic_feature = torch.tensor(atomic_feature, dtype=torch.float)
            molecular_feature = torch.tensor(molecular_feature, dtype=torch.float).view(1, -1)
            fingerprint = torch.tensor(fingerprint, dtype=torch.float).view(1, -1)
            MLM = torch.tensor(MLM, dtype=torch.float).view(1, -1)
            HLM = torch.tensor(HLM, dtype=torch.float).view(1, -1)
            y = torch.concat([MLM, HLM], dim=1)
            
            return Data(x=atomic_feature, mol_f=molecular_feature, fp=fingerprint,
                    edge_index=edge_index, edge_attr=edge_attr, MLM=MLM, HLM=HLM)
        else:
            atomic_feature = torch.tensor(atomic_feature, dtype=torch.float)
            molecular_feature = torch.tensor(molecular_feature, dtype=torch.float).view(1, -1)
            fingerprint = torch.tensor(fingerprint, dtype=torch.float).view(1, -1)
            
            return Data(x=atomic_feature, mol_f=molecular_feature, fp=fingerprint,
                    edge_index=edge_index, edge_attr=edge_attr)
            
        
        
        
        
class KFold_pl_DataModule(pl.LightningDataModule):
    def __init__(self,
                 train_df: str = './new_train.csv',
                 k_idx: int =1, # fold index
                 num_split: int = 5, # fold number, if k=1 then return the whole data
                 split_seed: int = 42,
                 batch_size: int = 1, 
                 num_workers: int = 0,
                 pin_memory: bool = False,
                 persistent_workers: bool=True,
                 transform1=None,
                 transform2=None
                 ) -> None:
        super().__init__()
        persistent_workers = True if num_workers > 0 else False
        self.save_hyperparameters(logger=False)
        self.transform1 = transform1
        self.transform2 = transform2
        self.train_data = None
        self.val_data = None
        self.num_cls = 0

        self.setup()

    def setup(self, stage=None) -> None:
        if not self.train_data and not self.val_data:
            df = pd.read_csv(self.hparams.train_df, index_col=0)

            mask = df['AlogP'] != df['AlogP']
            df.loc[mask, 'AlogP'] = df.loc[mask, 'MolLogP']

            PandasTools.AddMoleculeColumnToFrame(df,'SMILES','Molecule')
            df["FPs"] = df.Molecule.apply(generator.get_molecule_fingerprints)
            train_fps = np.stack(df["FPs"])

            feature_selector1 = self.transform1
            feature_selector2 = self.transform2
            
            mol_f = feature_selector1.fit_transform(df[feature_label])
            fps = feature_selector2.fit_transform(train_fps)

            

            kf = KFold(n_splits=self.hparams.num_split,
                       shuffle=True,
                       random_state=self.hparams.split_seed)
            all_splits = [k for k in kf.split(df)]
            train_idx, val_idx = all_splits[self.hparams.k_idx]
            train_idx, val_idx = train_idx.tolist(), val_idx.tolist()


            train_df = df.iloc[train_idx]
            train_fp = fps[train_idx]
            train_mol_f = mol_f[train_idx]
            
            val_df = df.iloc[val_idx]
            val_fp = fps[val_idx]
            val_mol_f = mol_f[val_idx]
    

            
            self.train_data = Chemcial_dataset(data_frame=train_df, fps=train_fp, mol_f=train_mol_f, transform=None, is_train=True)
            self.val_data = Chemcial_dataset(data_frame=val_df, fps=val_fp, mol_f=val_mol_f, transform=None, is_train=True)


    def train_dataloader(self):
        return DataLoader(self.train_data,
                          batch_size=self.hparams.batch_size,
                          shuffle=True,
                          num_workers=self.hparams.num_workers,
                          persistent_workers=self.hparams.persistent_workers,
                          pin_memory=self.hparams.pin_memory,
                          drop_last=True)
                          
    def val_dataloader(self):
        return DataLoader(self.val_data,
                          batch_size=self.hparams.batch_size,
                          shuffle=False,
                          num_workers=self.hparams.num_workers,
                          persistent_workers=self.hparams.persistent_workers,
                          pin_memory=self.hparams.pin_memory)
    


if __name__ == '__main__':
    feature_selector1 = VarianceThreshold(threshold=0.05)
    feature_selector2 = VarianceThreshold(threshold=0.05)
    data = KFold_pl_DataModule(transform1=feature_selector1, transform2=feature_selector2)
    
    train_loader = data.train_dataloader()
    val_loader = data.val_dataloader()

    for batch in train_loader:
        # DataBatch(x=[29, 34], edge_index=[2, 62], edge_attr=[62], mol_f=[1, 36], fp=[5235], MLM=[1], HLM=[1], batch=[29], ptr=[2])
        print(batch)
        break

#         DataBatch(x=[22, 167], edge_index=[2, 48], edge_attr=[48], y=[1, 2], mol_f=[1, 7], fp=[1, 300], input_ids=[1, 26], MLM=[1, 1], HLM=[1, 1], batch=[22], ptr=[2])
#         DataBatch(x=[19, 167], edge_index=[2, 42], edge_attr=[42], mol_f=[27], fp=[5235], MLM=[1], HLM=[1], batch=[19], ptr=[2])

DataBatch(x=[21, 167], edge_index=[2, 48], edge_attr=[48], mol_f=[1, 27], fp=[1, 5235], MLM=[1, 1], HLM=[1, 1], batch=[21], ptr=[2])


In [6]:
for i in train_loader:
    print(i.fp.shape[1])
    print(i.mol_f.shape[1])
    input_size1 = i.fp.shape[1]
    input_size2 = i.mol_f.shape[1]
    break


5235
27


In [7]:
# Hyperparameter
CFG = {'BATCH_SIZE': 256,
       'EPOCHS': 200,
       'INPUT_SIZE1': input_size1,
       'INPUT_SIZE2': input_size2,
       'HIDDEN_SIZE': 1024,
       'OUTPUT_SIZE': 1,
       'DROPOUT_RATE': 0.9,
       'LEARNING_RATE': 1e-4}

In [8]:
test_df = pd.read_csv('./new_test.csv', index_col=0)

PandasTools.AddMoleculeColumnToFrame(test_df,'SMILES','Molecule')
test_df["FPs"] = test_df.Molecule.apply(generator.get_molecule_fingerprints)
test_fps = np.stack(test_df["FPs"])

test_mol_f = feature_selector1.transform(test_df[feature_label])
test_fps = feature_selector2.transform(test_fps)

test_idx = test_df.reset_index().index
test_idx = list(map(int, test_idx))


test_df = test_df.iloc[test_idx]
test_fp = test_fps[test_idx]
test_mol_f = test_mol_f[test_idx]

test_loader = Chemcial_dataset(data_frame=test_df, fps=test_fp, mol_f=test_mol_f, transform=None, is_train=False)




## Model

In [9]:
class Net(nn.Module):
    def __init__(self, input_size2, hidden_size, dropout_rate, output_size):
        super(Net, self).__init__()
        
        # fc 레이어 3개와 출력 레이어
        self.fc1 = nn.Linear(input_size2, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.fc_out = nn.Linear(hidden_size, output_size)
        
        # 정규화
        self.ln1 = nn.LayerNorm(hidden_size)
        self.ln2 = nn.LayerNorm(hidden_size)
        self.ln3 = nn.LayerNorm(hidden_size)        
        self.ln4 = nn.LayerNorm(hidden_size)
        
        # 활성화 함수
        self.activation = nn.ReLU()
        
        # Dropout
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):   
        out = self.fc1(x)
        out = self.ln1(out)
        out = self.activation(out)
        out = self.dropout(out)
        
        out = self.fc2(out)
        out = self.ln2(out)
        out = self.activation(out)
        out = self.dropout(out)
        
        out = self.fc3(out)
        out = self.ln3(out)
        out = self.activation(out)
        out = self.dropout(out)

        out = self.fc4(out)
        out = self.ln4(out)
        out = self.activation(out)
        out = self.dropout(out)
        
        out = self.fc_out(out)
        return out

In [10]:
class RNN(nn.Module):
    def __init__(self, input_size1, hidden_size, dropout_rate, output_size):
        super(RNN, self).__init__()
        
        # fc 레이어 3개와 출력 레이어
        self.fc1 = nn.Linear(input_size1, hidden_size)
        self.fc2 = nn.GRU(hidden_size, hidden_size)
        self.fc3 = nn.RNN(hidden_size, hidden_size)
        self.fc4 = nn.RNN(hidden_size, hidden_size)
        self.fc_out = nn.Linear(hidden_size, output_size)
        
        # 정규화
        self.ln1 = nn.LayerNorm(hidden_size)
        self.ln2 = nn.LayerNorm(hidden_size)
        self.ln3 = nn.LayerNorm(hidden_size)
        self.ln4 = nn.LayerNorm(hidden_size)
        
        # 활성화 함수
        self.activation = nn.ReLU()
        
        # Dropout
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.ln1(out)
        out = self.activation(out)
        out = self.dropout(out)
        
        out, _ = self.fc2(out)
        out = self.ln2(out)
        out = self.activation(out)
        out = self.dropout(out)
        
#         out, _ = self.fc3(out)
#         out = self.ln3(out)
#         out = self.activation(out)
#         out = self.dropout(out)

#         out, _ = self.fc4(out)
#         out = self.ln4(out)
#         out = self.activation(out)
#         out = self.dropout(out)
        
        out = self.fc_out(out)
        return out

In [11]:
class ClassificationModel(nn.Module):
    def __init__(self, input_size1, input_size2, hidden_size, drop_rate, output_size):
        super(ClassificationModel, self).__init__()
        self.RNN_extractor = RNN(input_size1, hidden_size, drop_rate, output_size)
        self.Net_extractor = Net(input_size2, hidden_size, drop_rate, output_size)
#         self.classifier = nn.Linear(in_features=hidden_size*2, out_features=output_size)
        self.classifier = nn.Linear(in_features=output_size*2, out_features=output_size)

    def forward(self, fp, others):
        RNN_feature = self.RNN_extractor(fp)
        Net_feature = self.Net_extractor(others)
        feature = torch.cat([RNN_feature, Net_feature], dim=-1)
        output = self.classifier(feature)
        return output

In [12]:
model_MLM = ClassificationModel(CFG['INPUT_SIZE1'],CFG['INPUT_SIZE2'],CFG['HIDDEN_SIZE'],CFG['DROPOUT_RATE'],CFG['OUTPUT_SIZE'])
model_HLM = ClassificationModel(CFG['INPUT_SIZE1'],CFG['INPUT_SIZE2'],CFG['HIDDEN_SIZE'],CFG['DROPOUT_RATE'],CFG['OUTPUT_SIZE'])

In [13]:
# scheduler CosineAnnealingWarmUpRestarts
import math
from torch.optim.lr_scheduler import _LRScheduler

class CosineAnnealingWarmUpRestarts(_LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        super(CosineAnnealingWarmUpRestarts, self).__init__(optimizer, last_epoch)
    
    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr)*self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (1 + math.cos(math.pi * (self.T_cur-self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch
                
        self.eta_max = self.base_eta_max * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [14]:
# criterion = nn.MSELoss()
optimizer_MLM = torch.optim.Adam(model_MLM.parameters(), lr=CFG['LEARNING_RATE'])
optimizer_HLM = torch.optim.Adam(model_HLM.parameters(), lr=CFG['LEARNING_RATE'])
# optimizer_MLM = torch.optim.SGD(model_MLM.parameters(), lr=CFG['LEARNING_RATE'], momentum=0.9)
# optimizer_HLM = torch.optim.SGD(model_HLM.parameters(), lr=CFG['LEARNING_RATE'], momentum=0.9)
scheduler_MLM = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_MLM, mode='min', factor=0.5, patience=5, threshold_mode='abs', min_lr=0, eps=1e-12, verbose=True)
scheduler_HLM = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_HLM, mode='min', factor=0.5, patience=5, threshold_mode='abs', min_lr=0, eps=1e-12, verbose=True)
# scheduler_MLM = CosineAnnealingWarmUpRestarts(optimizer_MLM, T_0=20, T_mult=1, eta_max=0.1,  T_up=2, gamma=0.5)
# scheduler_HLM = CosineAnnealingWarmUpRestarts(optimizer_HLM, T_0=20, T_mult=1, eta_max=0.1,  T_up=2, gamma=0.5)
#주기, 주기변화율, 최대lr, 초기활성ep, 진폭변화율

## Training

In [15]:
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()

    def forward(self, y_hat, y):
        loss = torch.sqrt(self.mse(y_hat,y))
        return loss

In [16]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.train()
    model.to(device)

    criterion = RMSELoss().to(device)
    
    
    for epoch in range(CFG['EPOCHS']):
        running_loss = 0
        for loader in train_loader:
            optimizer.zero_grad()
            inputs = loader.fp.to(device)
            others = loader.mol_f.to(device)
            if model==model_MLM:
                targets = loader.MLM.to(device)
            else:
                targets = loader.HLM.to(device)
            
            output = model(inputs, others)
            loss = criterion(output, targets)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            

            
        if epoch % 10 == 0:
            
            val_loss = validation(model, criterion, val_loader, device)
            
            print(f'Epoch: {epoch}, Train Loss: {running_loss/len(train_loader)}, Valid Loss: {val_loss/len(val_loader)}')
            
            model.train()
        if scheduler is not None:
            scheduler.step(val_loss)
#             if scheduler.num_bad_epochs > scheduler.patience:
#                 print(f'Early stopping at epoch {epoch}...')
#                 break
    return model

In [17]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for loader in train_loader:
            inputs = loader.fp.to(device)
            others = loader.mol_f.to(device)
            if model==model_MLM:
                targets = loader.MLM.to(device)
            else:
                targets = loader.HLM.to(device)
            
            output = model(inputs, others)
            loss = criterion(output, targets)
            val_loss += loss.item()
            
    
    return val_loss

In [18]:
print("Training Start: MLM")
model_MLM = train(model_MLM, optimizer_MLM, train_loader, val_loader, scheduler_MLM, device)
print("Training Start: HLM")
model_HLM = train(model_HLM, optimizer_HLM, train_loader, val_loader, scheduler_HLM, device)

Training Start: MLM
Epoch: 0, Train Loss: 33.17528882665324, Valid Loss: 126.15233260699681
Epoch 00007: reducing learning rate of group 0 to 5.0000e-05.
Epoch: 10, Train Loss: 20.109169371126537, Valid Loss: 72.82078921130726
Epoch 00017: reducing learning rate of group 0 to 2.5000e-05.
Epoch: 20, Train Loss: 12.04138959802586, Valid Loss: 39.27342323802944
Epoch 00027: reducing learning rate of group 0 to 1.2500e-05.
Epoch: 30, Train Loss: 8.87753287584037, Valid Loss: 25.212926900535823
Epoch 00037: reducing learning rate of group 0 to 6.2500e-06.
Epoch: 40, Train Loss: 7.7293169177706496, Valid Loss: 19.452408308706115
Epoch 00047: reducing learning rate of group 0 to 3.1250e-06.
Epoch: 50, Train Loss: 7.3434777118923344, Valid Loss: 16.288244823685716
Epoch 00057: reducing learning rate of group 0 to 1.5625e-06.
Epoch: 60, Train Loss: 7.003577611464381, Valid Loss: 14.92372656976538
Epoch 00067: reducing learning rate of group 0 to 7.8125e-07.
Epoch: 70, Train Loss: 6.769226042141

## Inference

In [19]:
def inference(test_loader, model):
    model.eval()
    preds = []
    
    with torch.no_grad():
        for loader in test_loader:
            inputs = loader.fp.to(device)
            others = loader.mol_f.to(device)
            output = model(inputs, others)
            preds.extend(output.cpu().numpy().flatten().tolist())
    
    return preds

In [20]:
predictions_MLM = inference(test_loader, model_MLM)
predictions_HLM = inference(test_loader, model_HLM)

## Submission

In [21]:
submission = pd.read_csv('./sample_submission.csv')
submission

,id,MLM,HLM
0,TEST_000,0,0
1,TEST_001,0,0
2,TEST_002,0,0
3,TEST_003,0,0
4,TEST_004,0,0
...,...,...,...
478,TEST_478,0,0
479,TEST_479,0,0
480,TEST_480,0,0
481,TEST_481,0,0


In [22]:
submission['MLM'] = predictions_MLM
submission['HLM'] = predictions_HLM
submission

,id,MLM,HLM
0,TEST_000,26.235308,57.778999
1,TEST_001,78.703026,72.996941
2,TEST_002,38.237705,46.721073
3,TEST_003,41.198387,69.166420
4,TEST_004,38.166775,66.412666
...,...,...,...
478,TEST_478,22.717974,66.544838
479,TEST_479,76.779686,93.981628
480,TEST_480,29.285145,49.437218
481,TEST_481,73.572464,75.679611


In [24]:
submission.to_csv('baseline_submission.csv', index=False)